## そもそもARモデルをしっかり学習できるのかを確かめる

次時刻予測をさせる

In [12]:
import argparse
import os
path = os.getcwd()
path=path[:path.find('timeseries-WGAN')+15]
import numpy as np
from scipy import stats
import math
import sys
sys.path.append(path+"/")
import random
import statsmodels.api as sm
from scipy.stats import norm

import torchvision.transforms as transforms
from torchvision.utils import save_image

import torch.nn as nn
import torch.nn.functional as F
import torch
from torch.autograd import Variable

In [13]:
# 人工データを生成してくれる機械が置いてあるところ
import tsModel
# 学習用のニューラルネットが置いてあるところ
import models

In [14]:
%matplotlib inline
import matplotlib.pyplot as plt

In [15]:
# フォルダを作成（既にあるならそれで良し）
os.makedirs("output-images", exist_ok=True)
os.makedirs("parameters", exist_ok=True)

In [16]:
p=7

## NNで確かめる

In [17]:
i=0

In [18]:
net = models.LinearPredictNet(p = p, input_dim=1, is_bias=True)

In [35]:
! ls change_seed_ver2/parameters/

network_epoch100000_200000_batchSize64_networkSeed0_p7_networkBiasTrue_dataSeed7.pth
network_epoch10000_200000_batchSize64_networkSeed0_p7_networkBiasTrue_dataSeed7.pth
network_epoch10000_20000_batchSize64_networkSeed0_p7_networkBiasTrue_dataSeed0.pth
network_epoch10000_40000_batchSize64_networkSeed0_p7_networkBiasTrue_dataSeed0.pth
network_epoch1000_20000_batchSize64_networkSeed0_p7_networkBiasTrue_dataSeed0.pth
network_epoch1000_4000_batchSize64_networkSeed0_p7_networkBiasTrue_dataSeed0.pth
network_epoch110000_200000_batchSize64_networkSeed0_p7_networkBiasTrue_dataSeed7.pth
network_epoch11000_20000_batchSize64_networkSeed0_p7_networkBiasTrue_dataSeed0.pth
network_epoch120000_200000_batchSize64_networkSeed0_p7_networkBiasTrue_dataSeed7.pth
network_epoch12000_20000_batchSize64_networkSeed0_p7_networkBiasTrue_dataSeed0.pth
network_epoch130000_200000_batchSize64_networkSeed0_p7_networkBiasTrue_dataSeed7.pth
network_epoch13000_20000_batchSize64_networkSeed0_p7_networkBiasTrue_dataSeed0.pt

In [31]:
parameter_path = "change_seed_ver2/parameters/network_epoch40000_batchSize64_networkSeed0_p7_networkBiasTrue_dataSeed{0}.pth".format(i)
parameter_path

'change_seed_ver2/parameters/network_epoch40000_batchSize64_networkSeed0_p7_networkBiasTrue_dataSeed0.pth'

In [32]:
net.load_state_dict(torch.load(parameter_path))

<All keys matched successfully>

In [33]:
net.state_dict()

OrderedDict([('fc1.weight',
              tensor([[ 0.1394, -0.0615,  0.5654, -0.5325,  0.2241, -0.3913,  0.2614, -0.0314]])),
             ('fc1.bias', tensor([0.0573]))])

In [34]:
[0.3, -0.4, 0.2, -0.5, 0.6, -0.1, 0.1]

[0.3, -0.4, 0.2, -0.5, 0.6, -0.1, 0.1]

In [29]:
net.state_dict()["fc1.weight"][0]

tensor([ 0.1423, -0.0794,  0.5891, -0.5498,  0.2060, -0.4104,  0.2674,  0.0481])

In [12]:
net.state_dict()["fc1.bias"].item()

0.055861249566078186

In [13]:
params = []
mu = []

for i in range(10):
    net = models.LinearPredictNet(p = p, input_dim=1, is_bias=True)
    parameter_path = "change_seed/parameters/network_epoch20000_batchSize64_networkSeed0_p7_networkBiasTrue_dataSeed{0}.pth".format(i)
    net.load_state_dict(torch.load(parameter_path))
    params.append(np.array(net.state_dict()["fc1.weight"][0]))
    mu.append(net.state_dict()["fc1.bias"].item())

In [14]:
params

[array([ 0.31314477, -0.36504686,  0.27184433, -0.43856618,  0.33677167,
        -0.42010248,  0.26005104, -0.01064355], dtype=float32),
 array([ 0.29983357, -0.3969987 ,  0.27157873, -0.38687065,  0.2769836 ,
        -0.3992239 ,  0.2647001 ,  0.01933478], dtype=float32),
 array([ 0.27080438, -0.39342478,  0.28358912, -0.40527612,  0.22151943,
        -0.41049236,  0.2654277 ,  0.0069396 ], dtype=float32),
 array([ 0.29360873, -0.44982007,  0.2733666 , -0.43622828,  0.27736217,
        -0.42202893,  0.27404442, -0.00865356], dtype=float32),
 array([ 0.2563577 , -0.38253444,  0.32640874, -0.47440857,  0.26235667,
        -0.3748935 ,  0.30355993, -0.00252023], dtype=float32),
 array([ 0.32526284, -0.35008043,  0.2832891 , -0.35309443,  0.29167408,
        -0.37545577,  0.29418257, -0.01228052], dtype=float32),
 array([ 0.23107277, -0.40981576,  0.25780845, -0.38802522,  0.26731107,
        -0.37267247,  0.26718327,  0.00619552], dtype=float32),
 array([ 0.32252413, -0.40755153,  0.3034

In [15]:
mu

[0.055861249566078186,
 0.0794120505452156,
 -0.09066326916217804,
 -0.09570492804050446,
 0.016815723851323128,
 0.0342823751270771,
 0.022684520110487938,
 0.02085406892001629,
 0.07357200980186462,
 -0.03404462710022926]

In [16]:
print(np.array(params).mean(axis = 0))
print(np.array(params).std(axis = 0))

[ 0.28889686 -0.39398235  0.28126055 -0.40571442  0.2783966  -0.3943136
  0.28805017  0.00151257]
[0.02822203 0.02613973 0.01899416 0.03525268 0.02718427 0.01918498
 0.02692717 0.01054511]


In [17]:
print(np.array(mu).mean())
print(np.array(mu).std())

0.008306917361915111
0.05924134751805972


## 最小二乗法で確かめる 最尤推定と同じ？

In [19]:
Data = tsModel.SARIMA(a=[0.3,-0.4,0.3,-0.4,0.3,-0.4,0.3], N=1400, random_seed=0, sigma=2)
Data = torch.tensor(Data, dtype=torch.float)
Data = torch.tensor(Data)
Data=Data.view(1,-1)
trainData = Data[:,:1000]

/home/lab/ryuichi/.pyenv/versions/anaconda3-4.3.1/envs/test/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  This is separate from the ipykernel package so we can avoid doing imports until


In [20]:
ar_data = np.array(trainData.view(-1))
model = sm.tsa.AR(ar_data)
result = model.fit(maxlag=p)
print(result.params)
print(result.sigma2)

[ 0.05453728  0.2623633  -0.42140758  0.32590541 -0.43224141  0.27970304
 -0.3663014   0.30380131]
3.668713789131543


In [21]:
params = []
sigma2 = []
for i in range(10):
    Data = tsModel.SARIMA(a=[0.3,-0.4,0.3,-0.4,0.3,-0.4,0.3], N=1400, random_seed=i, sigma=2)
    Data = torch.tensor(Data, dtype=torch.float)
    Data = torch.tensor(Data)
    Data=Data.view(1,-1)
    trainData = Data[:,:1000]

    ar_data = np.array(trainData.view(-1))
    model = sm.tsa.AR(ar_data)
    result = model.fit(maxlag=p)
    params.append(result.params)
    sigma2.append(result.sigma2)

/home/lab/ryuichi/.pyenv/versions/anaconda3-4.3.1/envs/test/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


In [22]:
np.array(params).mean(axis=0)

array([ 0.00967116,  0.28675394, -0.38915383,  0.27312712, -0.4003957 ,
        0.28039884, -0.39498306,  0.28428313])

In [23]:
np.array(params).std(axis=0)

array([0.06404926, 0.025956  , 0.02070671, 0.02824756, 0.03074804,
       0.02224957, 0.02063428, 0.02866665])

In [24]:
np.array(sigma2).mean(axis=0)

3.887592035740114

In [25]:
np.sqrt(sigma2).mean()

1.9711659276568496

In [26]:
np.sqrt(sigma2).std()

0.04579215418415814

## change_seed_ver2で学習したネットワークのパラメータを確認

In [6]:
i=7

In [7]:
! ls change_seed_ver2/parameters/

network_epoch100000_200000_batchSize64_networkSeed0_p7_networkBiasTrue_dataSeed7.pth
network_epoch10000_200000_batchSize64_networkSeed0_p7_networkBiasTrue_dataSeed7.pth
network_epoch10000_20000_batchSize64_networkSeed0_p7_networkBiasTrue_dataSeed0.pth
network_epoch1000_20000_batchSize64_networkSeed0_p7_networkBiasTrue_dataSeed0.pth
network_epoch110000_200000_batchSize64_networkSeed0_p7_networkBiasTrue_dataSeed7.pth
network_epoch11000_20000_batchSize64_networkSeed0_p7_networkBiasTrue_dataSeed0.pth
network_epoch120000_200000_batchSize64_networkSeed0_p7_networkBiasTrue_dataSeed7.pth
network_epoch12000_20000_batchSize64_networkSeed0_p7_networkBiasTrue_dataSeed0.pth
network_epoch130000_200000_batchSize64_networkSeed0_p7_networkBiasTrue_dataSeed7.pth
network_epoch13000_20000_batchSize64_networkSeed0_p7_networkBiasTrue_dataSeed0.pth
network_epoch140000_200000_batchSize64_networkSeed0_p7_networkBiasTrue_dataSeed7.pth
network_epoch14000_20000_batchSize64_networkSeed0_p7_networkBiasTrue_dataSeed

In [8]:
parameter_path = "change_seed_ver2/parameters/network_epoch200000_batchSize64_networkSeed0_p7_networkBiasTrue_dataSeed{0}.pth".format(i)
parameter_path

'change_seed_ver2/parameters/network_epoch200000_batchSize64_networkSeed0_p7_networkBiasTrue_dataSeed7.pth'

In [9]:
# models.LinearPredictNet(p=hat_p, input_dim=1, is_bias=True)
net = models.LinearPredictNet(p = p, input_dim=1, is_bias=True)

In [10]:
net.load_state_dict(torch.load(parameter_path))

<All keys matched successfully>

In [11]:
net.state_dict()["fc1.weight"][0]

tensor([ 0.1168, -0.0907,  0.6041, -0.4822,  0.1798, -0.4074,  0.3503, -0.0297])

In [47]:
net.state_dict()["fc1.bias"].item()

0.0038742783945053816

In [48]:
[0.3, -0.4, 0.2, -0.5, 0.6, -0.1, 0.1]

[0.3, -0.4, 0.2, -0.5, 0.6, -0.1, 0.1]

In [39]:
params = []
mu = []

for i in range(10):
    net = models.LinearPredictNet(p = p, input_dim=1, is_bias=True)
    parameter_path = "change_seed_ver2/parameters/network_epoch20000_batchSize64_networkSeed0_p7_networkBiasTrue_dataSeed{0}.pth".format(i)
    net.load_state_dict(torch.load(parameter_path))
    params.append(np.array(net.state_dict()["fc1.weight"][0]))
    mu.append(net.state_dict()["fc1.bias"].item())

print(np.array(params).mean(axis = 0))
print(np.array(params).std(axis = 0))

print(np.array(mu).mean())
print(np.array(mu).std())


[ 0.02874842 -0.07622646  0.5611625  -0.4859038   0.14753151 -0.35869732
  0.2780871   0.00105612]
[0.20362435 0.09024195 0.198735   0.05197975 0.13810144 0.12194191
 0.11677206 0.01688177]
0.012232288299128414
0.07086198724015266
